> 要進raw頁面再wget，否則是下再網頁(html)

!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
q = 'the course has aleardy started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {"question": 3.0, "section": 0.5}
results = index.search(
    query=q,
    boost_dict= boost,
    num_results = 5
)

In [10]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [11]:
results = index.search(
    query=q,
    boost_dict= boost,
    filter_dict = {'course':'data-engineering-zoomcamp'},
    num_results = 5
)

In [12]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## 1.4 用GPT來整理輸出答案

In [13]:
from openai import OpenAI

In [14]:
client = OpenAI()

In [36]:
q

'the course has aleardy started, can I still enroll?'

In [16]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": q}]
)

In [20]:
response.choices[0].message.content

'It depends on the specific policies of the institution offering the course. Some institutions may allow late enrollment under certain circumstances, while others may have strict enrollment deadlines. It is best to contact the institution or instructor directly to inquire about late enrollment options.'

In [70]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION base on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEST when answering the QUESTION.


QUESTION: {question}
CONTEST: {context}
""".strip()

In [71]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [72]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    

In [73]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [74]:
prompt = prompt_template.format(question=q, context=context).strip()

In [75]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION base on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEST when answering the QUESTION.


QUESTION: the course has aleardy started, can I still enroll?
CONTEST: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related qu

### 這次使用上下文參考

In [76]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

'Based on the information provided in the FAQ database, yes, you can still enroll in the course even after it has already started. You will still be eligible to submit the homeworks, but make sure to be aware of the deadlines for turning in the final projects. It is recommended not to leave everything for the last minute.'